In [1]:
import pyodbc
import pandas as pd

conn_str = (
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=MSC-RANGE-KOLK;"
    r"DATABASE={Wrench Enterprise};"
    r"UID=wrench;"
    r"PWD=wrench@123;"
)

conn = pyodbc.connect(conn_str)
print("✅ Connected to MSSQL using SQL Authentication")

query_1 = "SELECT * FROM [dbo].[NCR_Report_updated]"  
df_NCR = pd.read_sql(query_1, conn)

query_2 = "SELECT * FROM [dbo].[FCD_Report_Internship_Siddhartha]"  
df_FCD = pd.read_sql(query_2, conn)

df_NCR['DOC_STATUS'] = df_NCR['DOC_STATUS'].replace({
    '0': 'Work In Progress',
    '3': 'Approved',
    '4': None  
})
df_NCR = df_NCR[df_NCR['DOC_STATUS'].notna()]

df_FCD['DOC_Status'] = df_FCD['DOC_Status'].replace({
    0: 'Work In Progress',
    3: 'Approved',
    4: None  
})
df_FCD = df_FCD[df_FCD['DOC_Status'].notna()]

df_FCD.fillna("", inplace=True)
df_FCD.columns = df_FCD.columns.str.strip()

from datetime import datetime
df_FCD['Ongoing_Delay_Days'] = (pd.Timestamp.today() - df_FCD['Current_Stage_Activation_Date']).dt.days

df_NCR.fillna("", inplace=True)
df_NCR.columns = df_NCR.columns.str.strip()

✅ Connected to MSSQL using SQL Authentication


C:\Temp\ipykernel_6088\2736836848.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_NCR = pd.read_sql(query_1, conn)
C:\Temp\ipykernel_6088\2736836848.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_FCD = pd.read_sql(query_2, conn)
C:\Temp\ipykernel_6088\2736836848.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_FCD.fillna("", inplace=True)
C:\Temp\ipykernel_6088\2736836848.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future v

In [2]:
def sanitize_dataframe(df):
    for col in df.columns:
        # Strip whitespace and replace empty strings with NaN
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.strip().replace('', pd.NA)
        
        # Try converting to numeric if it looks like it should be
        try:
            df[col] = pd.to_numeric(df[col], errors='ignore')
        except:
            pass
    return df


In [3]:
import pickle

# Sanitize both DataFrames
df_NCR = sanitize_dataframe(df_NCR)
df_FCD = sanitize_dataframe(df_FCD)

# Now save to parquet
df_NCR.to_parquet("ncr_data.parquet")
df_FCD.to_parquet("fcd_data.parquet")


C:\Temp\ipykernel_6088\32993171.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Temp\ipykernel_6088\32993171.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Temp\ipykernel_6088\32993171.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Temp\ipykernel_6088\32993171.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignor